In [1]:
# Import necessary libraries
import numpy as np, pandas as pd
import random
import time
import os
import more_itertools as mit

# Adjust options for viewing dataframes in notebook
pd.set_option('display.width', 2000)

In [2]:
# Function for reading a data file
def read_data(name):
    df = pd.read_csv('./../../data/clean_data/' + name + '.csv')
    df['Hour'] = pd.to_datetime(df['Hour'])
    df = df.set_index('Hour')
    df.index.freq = 'h'
    return df

In [3]:
# Function for locating random batches from dataset

## Take dataset, length of test data and number of batches to extract as parameters
## Find a number of digits that represent the indices of the original dataset, where the extracted batches will start from
## Each subset dataset will not intersect with other datasets

def extract_batches(df, test_length, batch_count):
    start = time.time()
    random_int_list = []
    m = 0
    random.seed(1)
    data_end = len(df)
    while m < batch_count:
        random_int = random.randint(0, data_end - test_length)
        q = 0
        for i in random_int_list:
            # Make sure batches do not overlap
            if i + test_length <= random_int or random_int + test_length <= i:
                pass
            else:
                q = 1
                # In case solution does not exist with current combination
                if time.time() - start > 2:
                    print('Restarted')
                    random_int_list = []
                    start = time.time()
                    m = 0

        # Prevent batches with 0 energy
        if df['Energy'][random_int: random_int + test_length].sum() == 0:
            q = 1

        # Record the position of the batch
        if q == 0:
            random_int_list.append(random_int)
            m = m + 1
    return random_int_list

In [4]:
# Range Pricing Method:
# The energy rate assigned to an hour's demand value is dependent on which demand threshold it falls under
# Demand thresholds are calculated by splitting the demand range evenly
# The higher the demand threshold the higher the price assigned

# Function for creating a price profile using the range pricing method

def range_pricing(test, min_price, max_price):

    # Possible prices per kWh are in range [min_price, max_price]
    price_set = [x / 100 for x in range(min_price, max_price + 1)]
    price_set_count = len(price_set)

    # The range of demand is split into equal bins, one for each possible price
    max_demand = max(test)
    min_demand = min(test)
    bin_size = (max_demand - min_demand) / price_set_count

    # Demand thresholds for each price are calculated
    demand_thresholds = []
    j = 1
    while j < price_set_count + 1:
        demand_thresholds.append(min_demand + bin_size * j)
        j = j + 1

    # To tackle float representation imprecision, the highest demand threshold is increased a little
    demand_thresholds[-1] = demand_thresholds[-1] + 1

    # Every demand value is compared with the demand thresholds and assigned a demand-proportional price
    price_profile = []
    i = 0
    while i < len(test):
        j = 0
        while j < price_set_count:
            if test[i] <= demand_thresholds[j]:
                price_profile.append(price_set[j])
                break
            j = j + 1
        i = i + 1

    return price_profile

In [5]:
# Percentile Pricing Method:
# The energy rate assigned to an hour's demand value is dependent on which demand threshold it falls under
# Demand thresholds are calculated by identifying demand values at specific percentile positions
# The higher the demand threshold the higher the price assigned

# Function for creating a price profile using the percentile pricing method

def percentile_pricing(test, min_price, max_price):

    # A list of possible prices in range [min_price, max_price] is created
    price_set = [x / 100 for x in range(min_price, max_price + 1)]
    price_set_count = len(price_set)

    # Percentile thresholds for each price are calculated
    bin_size = 100 / price_set_count
    percentile_thresholds = []
    j = 1
    while j < price_set_count + 1:
        percentile_thresholds.append(bin_size * j)
        j = j + 1

    # Demand thresholds for each price are calculated
    demand_thresholds = []
    for k in percentile_thresholds:
        demand_thresholds.append(np.percentile(test, k))

    # Every demand value is compared with the demand thresholds and assigned a demand-proportional price
    price_profile = []
    i = 0
    while i < len(test):
        j = 0
        while j < price_set_count:
            if test[i] <= demand_thresholds[j]:
                price_profile.append(price_set[j])
                break
            j = j + 1
        i = i + 1

    return price_profile

In [6]:
# Function for running the revenue simulation and collecting the results
def analyse(df, test_length, batch_count):
    # Find appropriate (random) start indices for batches to prevent overlap
    start_index_list = extract_batches(df, test_length, batch_count)
    
    # Create the empty results dataframe to hold the results
    df_results = pd.DataFrame(columns=['Energy (kWh)', 'Flat Energy', 'Time of Use', 'Range', 'Percentile'])

    for start_index in start_index_list:
        # Retrieve the batch test data
        test = df[start_index: start_index + test_length]

        # Calculate total energy demand in batch
        demand_value = np.round(test['Energy'].sum(), 2)

        # Calculate the total revenue for each pricing method: 
        ## Flat Energy Rate (0.23 dollars per kWh)
        flat_energy_value = np.round(test['Energy'].sum() * 0.23, 2)
        
        ## Time of Use Rate (0.25 dollars per kWh between 08:00 - 21:00, 0.2 dollars otherwise )
        peak_time = (test.index.hour <= 21) & (test.index.hour >= 8)
        tou_value = np.round(test[peak_time]['Energy'].sum() * 0.25 + test[~peak_time]['Energy'].sum() * 0.2, 2)

        # Call the new_method1 function to calculate revenue using NRange Pricing
        range_value = np.round(mit.dotproduct(range_pricing(test['Energy'], 21, 25), test['Energy']), 2)

        # Call the percentile_pricing function to calculate revenue using Percentile Pricing
        percentile_value = np.round(mit.dotproduct(percentile_pricing(test['Energy'], 21, 25), test['Energy']), 2)

        # Append the batch results to the results dataframe
        row = [demand_value, flat_energy_value, tou_value, range_value, percentile_value]
        df_results = pd.concat([df_results, pd.DataFrame([row], columns=df_results.columns)], ignore_index=True)

    # Calculate the average of each column
    average_value_list = np.round(df_results.mean(), 2).tolist()
    # Calculate the minimum of each column
    min_value_list = np.round(df_results.min(), 2).tolist()
    # Calculate the maximum of each column
    max_value_list = np.round(df_results.max(), 2).tolist()
    # Calculate the standard deviation of each column
    stddev_value_list = np.round(df_results.std(), 2).tolist()

    # Append the average to each column
    df_results = pd.concat([df_results, pd.DataFrame([average_value_list], columns=df_results.columns)], ignore_index=True)
    # Append the minimum to each column
    df_results = pd.concat([df_results, pd.DataFrame([min_value_list], columns=df_results.columns)], ignore_index=True)
    # Append the maximum to each column
    df_results = pd.concat([df_results, pd.DataFrame([max_value_list], columns=df_results.columns)], ignore_index=True)
    # Append the standard deviation to each column
    df_results = pd.concat([df_results, pd.DataFrame([stddev_value_list], columns=df_results.columns)], ignore_index=True)

    # Reformat the results dataframe
    df_results = df_results.transpose()
    df_results.columns = ['Batch 1', 'Batch 2', 'Batch 3', 'Batch 4', 'Batch 5', 'Batch 6', 'Batch 7', 'Batch 8', 'Batch 9', 'Batch 10',
                            'Average', 'Minimum', 'Maximum', 'Std. Dev.']
    
    # Display results
    display(df_results)

    return df_results

In [7]:
# Conducting extensive batch testing of revenue for different periods, with 10 random batches each
def main():
    # Make the directory for the results
    if os.path.isdir('../../data/revenue_results'):
        pass
    else:
        os.mkdir('../../data/revenue_results')

    print('-- Revenue simulation for different pricing methods --')
    for name in ['WEBSTER', 'CAMBRIDGE', 'MPL', 'RINCONADA LIB', 'TED THOMPSON', 'HAMILTON', 'HIGH', 'BRYANT']:

        df = read_data(name)
        print('\n{}'.format(name))
        file_name = '../../data/revenue_results/' + name + '.xlsx'

        # Write the tables to appropriate xlsx files
        with pd.ExcelWriter(file_name, mode='w', engine='xlsxwriter') as writer:
            
            # Modify execution parameters if needed
            test_length = [672, 336, 168, 72, 48, 24]                                     # Hours in batch
            batch_count = [10, 10, 10, 10, 10, 10]                                        # Batch count
            start_row = [0, 7, 14, 21, 28, 35]                                            # Row position in xlsx file to write results in
            durations = ['28 Days', '14 Days', '7 Days', '3 Days', '2 Days', '1 Day']     # Durations of batches, used for titles

            all_results_list = []

            i = 0
            while i < len(test_length):
                print('{} Revenue Simulation:'.format(durations[i]))

                # Run set of batches and collect results
                df_results = analyse(df, test_length[i], batch_count[i])
                all_results_list.append(df_results)

                # Write results to xlsx file
                df_results.to_excel(writer, sheet_name=name, startrow=start_row[i])
                pd.DataFrame(['{} Revenue'.format(durations[i])]).to_excel(writer, sheet_name=name, startrow=start_row[i], header=None, index=None)
                i = i + 1

            # Create summary dataframes
            df_summary_avg = pd.concat([df['Average'] for df in all_results_list], axis="columns")
            df_summary_min = pd.concat([df['Minimum'] for df in all_results_list], axis="columns")
            df_summary_max = pd.concat([df['Maximum'] for df in all_results_list], axis="columns")
            df_summary_stddev = pd.concat([df['Std. Dev.'] for df in all_results_list], axis="columns")

            for i in [df_summary_avg, df_summary_min, df_summary_max, df_summary_stddev]:
                i.columns = durations

            # Modify summary parameters if needed
            start_row = [42, 49, 56, 63]                                                        # Row position in xlsx file to write results in
            summary_name = [df_summary_avg, df_summary_min, df_summary_max, df_summary_stddev]  # Summary reference list
            print_title = ['Average', 'Minimum', 'Maximum', 'Standard Deviation']               # Title for printing results
            summary_title = ['Avg.', 'Min.', 'Max.', 'Std.Dev.']                                # Dataframe title in xlsx file

            i = 0
            while i < 4:
                print('{} Values Summary - {}:'.format(print_title[i], name))

                # Display summary
                display(summary_name[i])

                # Write summary to xlsx file
                summary_name[i].to_excel(writer, sheet_name=name, startrow=start_row[i])
                pd.DataFrame(['{} Summary'.format(summary_title[i])]).to_excel(writer, sheet_name=name, startrow=start_row[i], header=None, index=None)
                i = i + 1

            # Adjust the first column's width
            worksheet = writer.sheets[name]
            workbook = writer.book
            worksheet.set_column(0, 0, 15)
            # Adjust cell formating
            format1 = workbook.add_format({'bg_color': '#EDEDED', 'num_format': '#,##0.00', 'border': 1})
            worksheet.conditional_format(0, 0, 68, 14, {'type': 'no_blanks', 'format': format1})
main()

-- Revenue simulation for different pricing methods --

WEBSTER
28 Days Revenue Simulation:


,Batch 1,Batch 2,Batch 3,Batch 4,Batch 5,Batch 6,Batch 7,Batch 8,Batch 9,Batch 10,Average,Minimum,Maximum,Std. Dev.
Energy (kWh),6461.36,4470.50,5145.90,4512.07,7848.19,4678.38,7806.77,4569.64,1338.02,2650.73,4948.16,1338.02,7848.19,2050.92
Flat Energy,1486.11,1028.21,1183.56,1037.78,1805.08,1076.03,1795.56,1051.02,307.74,609.67,1138.08,307.74,1805.08,471.71
Time of Use,1571.19,1101.40,1264.40,1111.18,1897.79,1151.60,1875.70,1131.63,332.35,647.46,1208.47,332.35,1897.79,490.74
Range,1491.77,1023.26,1192.89,1038.14,1835.31,1075.09,1811.30,1061.75,298.30,614.79,1144.26,298.30,1835.31,480.64
Percentile,1561.22,1092.06,1250.75,1101.00,1885.23,1140.57,1870.22,1116.96,331.85,650.82,1200.07,331.85,1885.23,487.51


14 Days Revenue Simulation:


,Batch 1,Batch 2,Batch 3,Batch 4,Batch 5,Batch 6,Batch 7,Batch 8,Batch 9,Batch 10,Average,Minimum,Maximum,Std. Dev.
Energy (kWh),3136.17,1860.31,2711.74,2355.10,3878.36,1997.78,3893.65,2421.25,715.71,2434.13,2540.42,715.71,3893.65,951.72
Flat Energy,721.32,427.87,623.70,541.67,892.02,459.49,895.54,556.89,164.61,559.85,584.30,164.61,895.54,218.90
Time of Use,761.70,458.00,666.82,578.23,937.06,489.01,935.72,600.04,177.32,598.14,620.20,177.32,937.06,227.17
Range,724.66,426.23,630.37,547.98,922.63,452.88,906.45,564.81,160.35,563.08,589.94,160.35,922.63,227.03
Percentile,758.85,454.87,658.27,574.20,929.99,487.34,934.37,590.89,177.42,592.15,615.84,177.42,934.37,226.07


7 Days Revenue Simulation:


...

,28 Days,14 Days,7 Days,3 Days,2 Days,1 Day
Energy (kWh),1372.00,756.53,366.16,162.50,112.88,60.04
Flat Energy,315.56,174.00,84.22,37.38,25.96,13.81
Time of Use,337.70,187.12,90.74,40.35,27.95,14.74
Range,311.48,172.61,85.82,38.12,26.61,14.05
Percentile,336.35,185.08,89.70,39.77,27.61,14.61


Maximum Values Summary - BRYANT:


,28 Days,14 Days,7 Days,3 Days,2 Days,1 Day
Energy (kWh),6014.59,3204.29,1723.19,724.81,527.03,241.68
Flat Energy,1383.36,736.99,396.33,166.71,121.22,55.59
Time of Use,1488.90,793.15,426.65,179.14,129.97,60.33
Range,1386.32,744.74,404.17,169.54,124.16,56.78
Percentile,1471.55,784.13,422.05,176.38,128.05,58.94


Standard Deviation Values Summary - BRYANT:


,28 Days,14 Days,7 Days,3 Days,2 Days,1 Day
Energy (kWh),1582.24,783.02,449.87,202.51,133.98,64.73
Flat Energy,363.92,180.10,103.47,46.58,30.82,14.89
Time of Use,396.56,195.94,112.14,50.71,33.47,16.39
Range,362.69,181.55,105.08,47.43,31.46,15.23
Percentile,389.53,192.57,110.36,49.68,32.65,15.91
